<a href="https://colab.research.google.com/github/MRyo-ie/DataMining_Report3_Lv4_NLP_Models/blob/GoogleColab-merge/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report3

## Level 4: 文書分類せよ。


In [2]:
# 自分の Google Drive をマウント
# https://qiita.com/uni-3/items/201aaa2708260cc790b8#drive内のディレクトリをマウントする220180920
from google.colab import drive
import os
drive.mount('/content/gdrive')

#@markdown ### GitHubリポジトリをcloneして保存するパス （Google Drive上）
REPOSITORY_ROOT_DIR = 'Colab Notebooks/learning/old_nlp_techs' #@param {type: "string"}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### << 実行手順 >>

1.  ↑ の、このボタンを押す　→　　![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

1. ↖︎（左上）のどこかに「ドライブに保存」があるので、それをクリックして、  
  自分の Google Drive にこのファイルをコピーする。

1. ↑ の変数を設定する。
  - GitHub リポジトリをクローンする場所（GoogleDrive上の好きな場所）のパスを入力。

1. 実行。  
  - ↑で GoogleDrive をマウントするので、（英語の）指示に従って、  
    「ログイン」→「パスコードをコピー」→「四角枠に貼り付けしてEnter」

1. 後は勝手に実行してくれるはず

In [3]:
REPOSITORY_URL = 'https://github.com/MRyo-ie/DataMining_Report3_Lv4_NLP_Models.git'
DATA_DL_URL = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
GDRIVE_ROOT = '/content/gdrive/My Drive/'
REPO_PATH = os.path.join(GDRIVE_ROOT, REPOSITORY_ROOT_DIR, REPOSITORY_URL.split('/')[-1][:-4])

# モデル関連のパス
## Sentence Piece
SP_MODEL_DIR = os.path.join(REPO_PATH, 'models', 'Tokenizer', 'SentencePiece')

# よく使うパス
DATA_DIR = os.path.join(REPO_PATH, 'datas', 'articles_livedoor')
TMP_DIR = os.path.join(DATA_DIR, 'tmp')
FILE_PATH = os.path.join(DATA_DIR, 'tmp', DATA_DL_URL.split('/')[-1])

print("""
------------------------------------------------
<<<  パス一覧  >>>
REPO_PATH = {}

DATA_DL_URL = {}
DATA_DIR = {}
TMP_DIR = {}
FILE_PATH = {}
------------------------------------------------""".format(REPO_PATH, DATA_DL_URL, DATA_DIR, TMP_DIR, FILE_PATH))


------------------------------------------------
<<<  パス一覧  >>>
REPO_PATH = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models

DATA_DL_URL = https://www.rondhuit.com/download/ldcc-20140209.tar.gz
DATA_DIR = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor
TMP_DIR = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp
FILE_PATH = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp/ldcc-20140209.tar.gz
------------------------------------------------


### 【準備】 GitHub clone

In [4]:
if not os.path.exists(REPO_PATH):
    repo_parent_path = os.path.join(GDRIVE_ROOT, REPOSITORY_ROOT_DIR)
    %cd  "$repo_parent_path"
    !git clone "$REPOSITORY_URL"  "$REPO_PATH"
    %cd "$REPO_PATH"
else:
    # pull -f （と同じ意味のコマンド）
    %cd "$REPO_PATH"
    # 最新の状態に更新（強制）
    ! git fetch origin master
    ! git reset --hard origin/master

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models
From https://github.com/MRyo-ie/DataMining_Report3_Lv4_NLP_Models
 * branch            master     -> FETCH_HEAD
HEAD is now at 7954c4b 【add】「WikiIterator を追加」


### 【準備】 データ
今回は livedoor の記事データセットを使用。


In [0]:
import configparser
import glob
import os, sys
import pandas as pd
import subprocess
import tarfile
from tqdm import tqdm
from urllib.request import urlretrieve

#### DL& 読み込み

In [0]:
# !mkdir -p  "$TMP_DIR"
# %cd  "$TMP_DIR"

# if not os.path.exists(FILE_PATH):
#     ! wget  "$DATA_DL_URL"
# # 時間かかる。
# !tar  -zxvf   'ldcc-20140209.tar.gz'
# !ls

In [7]:
categories = [ 
    name for name 
    in os.listdir( os.path.join(TMP_DIR, "text") ) 
    if os.path.isdir( os.path.join(TMP_DIR, "text", name) ) ]

categories = sorted(categories)
categories

['dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news']

#### データの抽出・整備



- ラベル（記事の種類）一覧

In [8]:
table = str.maketrans({
    '\n': '',
    '\t': '　',
    '\r': '',
})

def extract_txt(filename):
    with open(filename) as text_file:
        # 0: URL, 1: timestamp
        text = text_file.readlines()[2:]
        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != '', text))
        return ''.join(text)

%cd  "$TMP_DIR/"

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp


In [9]:
%cd  "$TMP_DIR/"

# 割と時間かかる。
all_text = []
all_label = []
for cat in tqdm(categories):
    files = glob.glob(os.path.join("text", cat, "{}*.txt".format(cat)))
    files = sorted(files)
    body = [ extract_txt(elem).translate(table) for elem in files ]
    label = [cat] * len(body)
    
    all_text.extend(body)
    all_label.extend(label)

print(all_label)

  0%|          | 0/9 [00:00<?, ?it/s]

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp


100%|██████████| 9/9 [00:09<00:00,  1.07s/it]

['dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-tsushin', 'dokujo-t

- 特徴量とラベル

In [10]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})
df.head()

,text,label
0,友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月。独女の...,dokujo-tsushin
1,ネットで断ち切れない元カレとの縁携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だ...,dokujo-tsushin
2,相次ぐ芸能人の“すっぴん”披露 その時、独女の心境は？「男性はやっぱり、女性の“すっぴん”が...,dokujo-tsushin
3,ムダな抵抗！？ 加齢の現実ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「...,dokujo-tsushin
4,税金を払うのは私たちなんですけど！6月から支給される子ども手当だが、当初は子ども一人当たり月...,dokujo-tsushin


In [11]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)
df.head()

,text,label
0,新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子...,sports-watch
1,家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by lived...,kaden-channel
2,彼にあげたい韓国メンズコスメ、韓流俳優のような美肌男へ！年末の大イベント、クリスマスまであと...,peachy
3,快適なスマホライフのための必須アプリ「マトリックス レボリューションズ」(c)Warner ...,livedoor-homme
4,独女と上司の気になる関係人事異動の多い春は、職場の人間関係の悩みも増える時期。『an・an』...,dokujo-tsushin


#### ファイルに保存 (機械学習モデル用)

In [0]:
extract_dir = os.path.join(TMP_DIR, 'extracts')
SentencePiece_train_dir = os.path.join(SP_MODEL_DIR, 'tmp')
if not os.path.exists(os.path.join(SP_MODEL_DIR, 'tmp')):
    ! mkdir  "$extract_dir"
    ! mkdir  "$SentencePiece_train_dir"

dpaths = {
    'test' : os.path.join(extract_dir, "test.tsv"),
    'val' : os.path.join(extract_dir, "val.tsv"),
    'train' : os.path.join(extract_dir, "train.tsv"),
}

In [24]:
# Sentence Piece 用の raw text データを作成
df.to_csv( os.path.join(SentencePiece_train_dir, "train.txt"), sep=' ', index=False, header=False, columns=['text'] )
! head -3  "$SentencePiece_train_dir"/train.txt

# 言語モデル用の train, test, dev データファイルを作成
df[:len(df) // 5].to_csv( dpaths['test'], sep='\t', index=False )
df[len(df) // 5:len(df)*2 // 5].to_csv( dpaths['val'], sep='\t', index=False )
df[len(df)*2 // 5:].to_csv( dpaths['train'], sep='\t', index=False )
P = dpaths['test']
! head -3  "$P"

text	label
新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子100メートルバタフライの加藤ゆか（25歳）が2大会連続の五輪出場を決めた。57秒77と、自身が持つ日本新記録を更新しての五輪切符ゲットだ。前回大会の北京五輪選考ではガチガチに緊張していたという加藤は、幾多の経験を得て、強い精神力を培った。記録更新での五輪出場権獲得に、爽やかな笑顔で喜びを爆発させた。百花繚乱の日本女子競泳界の中でも、加藤は美女スイマーの筆頭に数えられる。目鼻立ちの整った顔に、白く透き通るような肌、そして鍛えあげられたアスリートの肉体に、ネット上での人気も上々だ。前回の北京五輪では予選敗退で悔し涙を呑んだ加藤。4年間の濃密な時間を経て、速く、より美しくなった「バタフライの女王」が、ロンドンのプールを沸かす。・加藤ゆか フォト	sports-watch
家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by livedoor ニュース」 【話題】NHN Japanは、同社が提供する「livedoorニュース」で配信している記事の中から、IT記事に特化したニュースをまとめて閲覧することができるスマートフォンアプリ「ITニュース by livedoor ニュース」（iOS・Android対応／無料）を公開している。好きなジャンルの情報を逃さない向かうところ敵なしのITニュースリーダーだ。というのも、このアプリではlivedoorニュースの編集部が厳選したIT記事が提供されるが、編集部の手作業によって「ITビジネス」「WEBサービス」「マーケティング」「モバイル」「デジタル家電」などの12のカテゴリに振り分けられるので、数ある情報の中から興味を持った分野のニュースのみをすばやくチェックできる。ITトレンドを見落としたくない人は今すぐダウンロードしよう。■快適操作で情報共有が簡単、雑誌感覚でスラスラ読める面白かったニュースはTwitterやFacebookなどソーシャル系ツールでの情報共有も可能。操作方法はアイコンをタッチするだけで行えるので簡単だ。また、記事ページを左右にフリックして次の記事を読むことができるので、雑誌感覚でスラスラ読める。記事ページ上のフォントサイズ用ボタンをタッチすることで文字サイズ変更も可能だ。■ラ

### Layer1 : 分かち書き（Tokenize）
- 今回は **Sentence Piece** を使ってみる。  
  https://www.madopro.net/entry/sentencepiece_rnn_lm

In [13]:
! pip install sentencepiece

In [25]:
%cd  "$SP_MODEL_DIR"
!echo '[Info] Sentence Piece の学習を開始します。'
if not os.path.exists('model'):
    ! mkdir  'model'

# SentencePiece を学習する：モデルを作る。
import sentencepiece as spm
# 5分くらいかかる。（コメントアウト外す）
spm.SentencePieceTrainer.Train('--input=tmp/train.txt --model_prefix=model/m --vocab_size=8000')

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/models/Tokenizer/SentencePiece
[Info] Sentence Piece の学習を開始します。


True

In [32]:
# 学習したSentencePieceモデルを読み込む
sp = spm.SentencePieceProcessor()
sp.Load('model/m.model')
print( sp.EncodeAsPieces('吾輩は猫である') )
print( sp.EncodeAsPieces(df.at[1, 'text']) )

['▁', '吾', '輩', 'は', '猫', 'である']
['▁', '家電', 'チャンネル', 'の', '記事', 'も', '配信', '!', '向', 'か', 'う', 'ところ', '敵', 'なし', 'の', 'スマホ', 'アプリ', '「', 'IT', 'ニュース', '▁by', '▁livedoor', '▁', 'ニュース', '」', '▁【', '話題', '】', 'N', 'H', 'N', '▁Japan', 'は', '、', '同社', 'が', '提供', 'する', '「', 'livedoor', 'ニュース', '」', 'で', '配信', 'している', '記事', 'の中から', '、', 'IT', '記事', 'に', '特', '化', 'した', 'ニュース', 'を', 'まとめて', '閲覧', 'することができる', 'スマートフォン', 'アプリ', '「', 'IT', 'ニュース', '▁by', '▁livedoor', '▁', 'ニュース', '」(', 'iOS', '・', 'Android', '対応', '/', '無料', ')', 'を', '公開', 'している', '。', '好きな', 'ジャンル', 'の', '情報を', '逃', 'さない', '向', 'か', 'う', 'ところ', '敵', 'なし', 'の', 'IT', 'ニュース', 'リーダー', 'だ', '。', 'という', 'の', 'も', '、', 'このアプリ', 'では', 'livedoor', 'ニュース', 'の', '編集部', 'が', '厳', '選', 'した', 'IT', '記事', 'が', '提供', 'される', 'が', '、', '編集部', 'の', '手', '作業', 'によって', '「', 'IT', 'ビジネス', '」「', 'WEB', 'サービス', '」「', 'マーケティング', '」「', 'モバイル', '」「', 'デジタル', '家電', '」', 'などの', '12', 'の', 'カテゴリ', 'に', '振り', '分け', 'られる', 'の', 'で', '、', '数', 'ある', '情報', 'の

### Layer2 : Embedding
- 今回は、BoW+TF-IDF、共起行列、XLNet の予定。

### Layer3 : 分類モデル
- 今回は、SVM で行う。

In [19]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import pandas as pd
import glob
import os
import time
import math

use_gpu = torch.cuda.is_available()
print('use_gpu : ', use_gpu)

use_gpu :  True


In [0]:
# wakati_corpus.txtから一部データを読み込む
class Corpus(object):
    def __init__(self, sp_loaded, dpaths):
        self.sp = sp_loaded

        self.train = self.load_from_file(dpaths['train'])
        self.valid = self.load_from_file(dpaths['val'])
        self.test = self.load_from_file(dpaths['test'])

        self.train = self.assign_ids(self.train)
        self.valid = self.assign_ids(self.valid)
        self.test = self.assign_ids(self.test)
    
    def load_from_file(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read().split('\n')
    
    def assign_ids(self, texts):
        tokens = []
        for text in texts:
            tokens.extend(self.sp.EncodeAsIds(text))
        
        ids = torch.from_numpy(np.array(tokens, dtype=np.int64))
        return ids

In [0]:
# コーパスを実際に読み込む
corpus = Corpus()